### Singleton Pattern
Many times we just need a single instance of a class. Everytime an instance is requested, the same object is returned thereby reducing object creation overhead. For example:
- Logging frameworks
- Java beans provided by Spring framework
- Runtime

### Implementation
The main step is to mark constructor as private so that new instance of the Singleton class cannot be instantiated using the new keyword. Instead provide a `getInstance` method

In [1]:
class Singleton{
    private static Singleton instance = new Singleton();
    
    private Singleton(){}
    
    public static Singleton getInstance(){
        return instance;
    }
}

In [2]:
Singleton s1 = Singleton.getInstance();
Singleton s2 = Singleton.getInstance();

if(s1 == s2)
    System.out.println("Same object");
else
    System.out.println("Different objects");

Same object


The above implementation is an example for eager initialisation, we have an instance whether we want it or not. Another alternative is to go for lazy initialized version, which can lead to performance gain

In [ ]:
class Singleton{
    private static Singleton instance;
    
    private Singleton(){}
    
    public static Singleton getInstance(){
        if(instance == null){
            instance = new Singleton();
        }
        
        return instance;
    }
}

What if multiple threads call getInstance method at the same time? The above code snippet is not thread safe, therefore instead of getting a single object, multiple different objects can get created.  
The eager initialization version can be made thread safe just by a small change:

In [ ]:
// Make the instance reference final
private static final Singleton instance = new Singleton();

To make the lazy initialized version thread safe, we can make the `getInstance` method synchronized. However, making the entire method synchronized has performance penalty specially if it will be called multiple times in multiple threads. Instead, we employ the following:

In [ ]:
class Singleton{
    private static volatile Singleton instance;
    
    private Singleton(){}
    
    public static Singleton getInstance(){
        if(instance == null){
            synchronized(Singleton.class){
                if(instance == null)
                    instance = new Singleton();
            }
        }
        
        return instance;
    }
}

Singleton class is now thread safe, however there is still a way to multiple different objects using reflection

In [6]:
import java.lang.reflect.Constructor;
    
class CreateObjects{
    public static void main(String[] args) throws Exception{
        Constructor<Singleton> constructor = Singleton.class.getDeclaredConstructor();
        constructor.setAccessible(true);
        Singleton obj1 = constructor.newInstance();
        Singleton obj2 = constructor.newInstance();
    }
}

To prevent this, we can add the following check to the constructor:

In [ ]:
private Singleton() {
    if(instance != null) {
        throw new RuntimeException("Cannot call constructor");
    }
}

However this still doesn't cover the case when the constructor is called before `getInstance`. Enums are yet another way to implement singleton pattern.